In [10]:
from autora.cycle.filesystem import FilesystemCycle
from sklearn.linear_model import LinearRegression
import numpy as np

In [11]:
def ground_truth(x):
    return x + 1
from autora.variable import VariableCollection, Variable
metadata_0 = VariableCollection(
   independent_variables=[Variable(name="x1", allowed_values=range(11))],
   dependent_variables=[Variable(name="y", value_range=(-20, 20))],
   )



In [12]:
from autora.experimentalist.pipeline import make_pipeline
example_experimentalist = make_pipeline(
    [metadata_0.independent_variables[0].allowed_values])

def get_example_synthetic_experiment_runner():
    rng = np.random.default_rng(seed=180)
    def runner(x):
        return ground_truth(x) + rng.normal(0, 0.1, x.shape)
    return runner
example_synthetic_experiment_runner = get_example_synthetic_experiment_runner()
example_synthetic_experiment_runner(np.ndarray([1]))



array([1.04339546])

In [13]:

example_theorist = LinearRegression()


In [14]:
from autora.cycle.result.serializer.yaml_ import YAMLResultCollectionSerializer
import pathlib

cycle = FilesystemCycle(
    metadata=metadata_0,
    theorist=example_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    monitor=lambda data: print(f"Generated {len(data)}-th datum, a new {data[-1].kind}"),
    serializer=YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting"))
)




In [15]:
cycle.run(10)


Generated 1-th datum, a new CONDITION
Generated 2-th datum, a new OBSERVATION
Generated 3-th datum, a new THEORY
Generated 4-th datum, a new CONDITION
Generated 5-th datum, a new OBSERVATION
Generated 6-th datum, a new THEORY
Generated 7-th datum, a new CONDITION
Generated 8-th datum, a new OBSERVATION
Generated 9-th datum, a new THEORY
Generated 10-th datum, a new CONDITION


In [16]:
new_cycle = FilesystemCycle(
    state = YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting")),
    theorist=example_theorist,
    experimentalist=example_experimentalist,
    experiment_runner=example_synthetic_experiment_runner,
    monitor=lambda data: print(f"Generated {len(data)}-th datum, a new {data[-1].kind}"),)
new_cycle.run(num_steps=10)


Generated 11-th datum, a new OBSERVATION
Generated 12-th datum, a new THEORY
Generated 13-th datum, a new CONDITION
Generated 14-th datum, a new OBSERVATION
Generated 15-th datum, a new THEORY
Generated 16-th datum, a new CONDITION
Generated 17-th datum, a new OBSERVATION
Generated 18-th datum, a new THEORY
Generated 19-th datum, a new CONDITION
Generated 20-th datum, a new OBSERVATION


In [18]:
new_cycle.serializer = YAMLResultCollectionSerializer(pathlib.Path("./saving_restarting"))
new_cycle.dump()
